In [1]:
import json
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline

In [2]:
data = json.load(open('../data/ia_doc2vec_clustering.json'))
print(data.keys())

dict_keys(['n_clusters', 'true_labels', 'node_ids', 'kmeans_labels', 'hierarchical_labels'])


In [3]:
from sklearn import metrics
from gensim import models

C:\Users\nebboy4\Anaconda3\envs\github_env\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
from doc2vec.doc2vec import doc2vec
from utilities import score_purity
from utilities import score_agreement

In [5]:
d2v_model = doc2vec(model=models.Doc2Vec.load("../data/scotus/scotus_model.doc2vec"),label_docs=False)

In [6]:
embeddings = d2v_model.model.docvecs

In [7]:
node_order = {}
node_ids = []

for i in range(len(data['node_ids'])):
    node_order[i] = data['node_ids'][i]
    node_ids.append(data['node_ids'][i])

X = [embeddings[tag].tolist() for tag in node_ids]

k_to_kmeans = [data['kmeans_labels'][data['node_ids'].index(n_id)] for n_id in node_ids]
k_to_hierarchical = [data['hierarchical_labels'][data['node_ids'].index(n_id)] for n_id in node_ids]
print('complete')

complete


In [8]:
scores = {}

scores['accuracy_hierarchical'] = metrics.accuracy_score(data['true_labels'], data['hierarchical_labels'])
scores['accuracy_kmeans'] = metrics.accuracy_score(data['true_labels'], data['kmeans_labels'])

scores['nmi_hierarchical'] = metrics.normalized_mutual_info_score(data['true_labels'], data['hierarchical_labels'])
scores['nmi_kmeans'] = metrics.normalized_mutual_info_score(data['true_labels'], data['kmeans_labels'])

scores['purity_hierarchical'] = score_purity(data['true_labels'], data['hierarchical_labels'])
scores['purity_kmeans'] = score_purity(data['true_labels'], data['kmeans_labels'])

scores['sil_hierarchical'] = metrics.silhouette_score(X, k_to_hierarchical)
scores['sil_kmeans'] = metrics.silhouette_score(X, k_to_kmeans)

#scores['agreement'][k] = score_agreement(v['kmeans_labels'], v['hierarchical_labels'])

print(scores)

{'accuracy_hierarchical': 0.16593150439304286, 'accuracy_kmeans': 0.038407746100053791, 'nmi_hierarchical': 0.20405893511016829, 'nmi_kmeans': 0.15600332421952043, 'purity_hierarchical': 0.5696611081226466, 'purity_kmeans': 0.22431414739107047, 'sil_hierarchical': 0.00084685531903025163, 'sil_kmeans': -0.041730772084732089}


In [12]:
sort_scores = [(k, scores[k]) for k in sorted(scores, key=scores.get, reverse=True)]

for s in sort_scores:
    print(s[0],'-'*(30-len(s[0])),s[1])

purity_hierarchical ----------- 0.5696611081226466
purity_kmeans ----------------- 0.22431414739107047
nmi_hierarchical -------------- 0.20405893511
accuracy_hierarchical --------- 0.165931504393
nmi_kmeans -------------------- 0.15600332422
accuracy_kmeans --------------- 0.0384077461001
sil_hierarchical -------------- 0.00084685531903
sil_kmeans -------------------- -0.0417307720847


# Classification

In [16]:
X_train = X[:int(len(X)*.8)]
print(len(X_train))
X_test = X[-int(len(X)-int(len(X)*.8)):]
print(len(X_test))

22308
5577


In [17]:
print(len(X))

27885
